<a href="https://colab.research.google.com/github/Raptor-sj22/ML-Training/blob/main/logistic-regression/Logistic_Regression_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code performs logistic regression with 3rd-degree polynomial features using TensorFlow/Keras to classify a synthetic 2D dataset. The input features are expanded using PolynomialFeatures from scikit-learn to capture non-linear relationships. A single-layer neural network with sigmoid activation and L2 regularization (λ = 0.001) is used as the classifier. The model is trained on 300 samples for 200 epochs using the Adam optimizer, binary crossentropy as the loss function, and reports accuracy as a metric. The decision boundary is visualized over a mesh grid, and an interactive threshold slider allows real-time updates to the classification regions.

In [13]:
#@title Model with a Data set with random pattern
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

from IPython.display import display

#commented this line for githup
#import ipywidgets as widgets

In [ ]:
#@title Model with a Data set with random pattern



# 1️⃣ Generate a synthetic 2D dataset
X, y = make_classification(
    n_samples=300, n_features=2,
    n_redundant=0, n_informative=2,
    n_clusters_per_class=1,
    class_sep=1.0, random_state=42
)

# 2️⃣ Add polynomial features
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X)

# 3️⃣ Define a simple regression-based classifier (logistic regression with L2 regularization)
model = Sequential([
    Dense(1,
          input_dim=X_poly.shape[1],
          activation='sigmoid',
          kernel_regularizer=regularizers.l2(.001))  # L2 regularization (lambda = 0.01)
])

# 4️⃣ Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 5️⃣ Train the model
model.fit(X_poly, y, epochs=200, verbose=1)

# 6️⃣ Visualize the dataset and decision boundary

# Create a mesh grid for plotting decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))

# Transform grid points to polynomial feature space
grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_points_poly = poly.transform(grid_points)

# Predict probabilities for each grid point
Z = model.predict(grid_points_poly)
Z = Z.reshape(xx.shape)

# Predict on training set once (probs are fixed)
probs = model.predict(X_poly)

def plot_with_threshold(threshold):
    # Predict over the mesh grid with current threshold
    Z_thresh = (Z > threshold).astype(int)

    plt.figure(figsize=(8,6))

    # Background region (prediction)
    plt.contourf(xx, yy, Z_thresh, levels=[-1, 0, 1], alpha=0.3, colors=['blue', 'orange'])

    # Decision boundary line
    plt.contour(xx, yy, Z, levels=[threshold], colors='k', linewidths=1)

    # Points: fixed color based on true label (y), NOT threshold
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', edgecolors='k', label='Class 0 (true)')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='orange', edgecolors='k', label='Class 1 (true)')

    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title(f"Decision Boundary (Threshold = {threshold:.2f})")
    plt.legend()
    plt.grid(True)
    plt.show()

# Interactive threshold slider
threshold_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Threshold:',
    continuous_update=True
)

# Display the interactive plot
widgets.interact(plot_with_threshold, threshold=threshold_slider)

In [ ]:
#@title Model with a Data set with Circular pattern
np.random.seed(42)
n_samples = 300

# Class 0: points inside the circle (radius < 1)
r0 = np.random.rand(n_samples // 2)
theta0 = 2 * np.pi * np.random.rand(n_samples // 2)
x0 = np.stack([r0 * np.cos(theta0), r0 * np.sin(theta0)], axis=1)

# Class 1: points outside the circle (radius between 1.5 and 2.5)
r1 = 1.5 + np.random.rand(n_samples // 2)
theta1 = 2 * np.pi * np.random.rand(n_samples // 2)
x1 = np.stack([r1 * np.cos(theta1), r1 * np.sin(theta1)], axis=1)

# Combine
X = np.vstack([x0, x1])
y = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))

# 2️⃣ Add polynomial features
poly = PolynomialFeatures(degree=4, include_bias=False)
X_poly = poly.fit_transform(X)

# 3️⃣ Define a simple regression-based classifier (logistic regression with L2 regularization)
model = Sequential([
    Dense(1,
          input_dim=X_poly.shape[1],
          activation='sigmoid',
          kernel_regularizer=regularizers.l2(0.001))
])

# 4️⃣ Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 5️⃣ Train the model
model.fit(X_poly, y, epochs=100, verbose=1)

# 6️⃣ Visualize the dataset and decision boundary

# Create a mesh grid for plotting decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))

# Transform grid points to polynomial feature space
grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_points_poly = poly.transform(grid_points)

# Predict probabilities for each grid point
Z = model.predict(grid_points_poly)
Z = Z.reshape(xx.shape)

# Predict on training set once (probs are fixed)
probs = model.predict(X_poly)

def plot_with_threshold(threshold):
    # Predict over the mesh grid with current threshold
    Z_thresh = (Z > threshold).astype(int)

    plt.figure(figsize=(8,6))

    # Background region (prediction)
    plt.contourf(xx, yy, Z_thresh, levels=[-1, 0, 1], alpha=0.3, colors=['blue', 'orange'])

    # Decision boundary line
    plt.contour(xx, yy, Z, levels=[threshold], colors='k', linewidths=1)

    # Points: fixed color based on true label (y), NOT threshold
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', edgecolors='k', label='Class 0 (true)')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='orange', edgecolors='k', label='Class 1 (true)')

    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title(f"Decision Boundary (Threshold = {threshold:.2f})")
    plt.legend()
    plt.grid(True)
    plt.show()

# Interactive threshold slider
threshold_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Threshold:',
    continuous_update=True
)

# Display the interactive plot
widgets.interact(plot_with_threshold, threshold=threshold_slider)